In [ ]:
!git clone -b lagrangian-ppo --single-branch https://github.com/nguyenduchuyiu/RL-for-5G-Energy-Saving.git

Cloning into 'RL-for-5G-Energy-Saving'...


In [ ]:
%cd RL-for-5G-Energy-Saving

In [ ]:
%%writefile app/train_scenarios/dense_urban.json
{
  "name": "Dense Urban",
  "description": "3GPP Dense urban with macro and micro cells",
  "deploymentScenario": "dense_urban",
  
  "carrierFrequency": 4e9,
  "systemBandwidth": 200e6,
  "layout": "two_layer",
  "isd": 200,
  
  "numSites": 7,
  "numSectors": 3,
  "antennaHeight": 25,
  "cellRadius": 200,
  
  "numUEs": 300,
  "userDistribution": "Uniform/macro",
  "ueSpeed": 3,
  "indoorRatio": 0.8,
  "outdoorSpeed": 30,
  
  "minTxPower": 10,
  "maxTxPower": 46,
  "basePower": 1000,
  "idlePower": 250,
  
  "simTime": 0,
  "timeStep": 1,
  
  "rsrpServingThreshold": -110,
  "rsrpTargetThreshold": -100,
  "rsrpMeasurementThreshold": -115,
  "dropCallThreshold": 1,
  "latencyThreshold": 50,
  "cpuThreshold": 95,
  "prbThreshold": 95,
  
  "trafficLambda": 20,
  "peakHourMultiplier": 1.3
}

Overwriting app/train_scenarios/dense_urban.json


In [1]:
%%writefile app/train_scenarios/urban_macro.json
    {
  "name": "3GPP Urban Macro",
  "description": "3GPP Urban macro deployment scenario with large cells and continuous coverage",
  "deploymentScenario": "urban_macro",
  
  "carrierFrequency": 2e9,
  "systemBandwidth": 100e6,
  "layout": "hexagonal_grid",
  "isd": 500,
  
  "numSites": 7,
  "numSectors": 3,
  "antennaHeight": 25,
  "cellRadius": 200,
  
  "numUEs": 300,
  "userDistribution": "mixed_outdoor_indoor",
  "ueSpeed": 30,
  "indoorRatio": 0.8,
  "outdoorSpeed": 30,
  
  "minTxPower": 20,
  "maxTxPower": 46,
  "basePower": 1000,
  "idlePower": 250,
  
  "simTime": 4096,
  "timeStep": 1,
  
  "rsrpServingThreshold": -110,
  "rsrpTargetThreshold": -100,
  "rsrpMeasurementThreshold": -115,
  "dropCallThreshold": 1,
  "latencyThreshold": 50,
  "cpuThreshold": 95,
  "prbThreshold": 95,
  
  "trafficLambda": 25,
  "peakHourMultiplier": 1.8
}

Overwriting app/train_scenarios/urban_macro.json


In [2]:
%%writefile app/train_scenarios/high_speed.json
{
  "name": "High Speed",
  "description": "3GPP High speed scenario for railway with linear deployment",
  "deploymentScenario": "high_speed",
  
  "carrierFrequency": 4e9,
  "systemBandwidth": 200e6,
  "layout": "linear_railway",
  "isd": 1732,
  
  "numSites": 7,
  "numSectors": 2,
  "antennaHeight": 35,
  "cellRadius": 1000,
  
  "numUEs": 300,
  "userDistribution": "100% of users in train",
  "ueSpeed": 500,
  "indoorRatio": 1.0,
  "trainLength": 200,
  "trackLength": 10000,
  
  "minTxPower": 20,
  "maxTxPower": 46,
  "basePower": 1200,
  "idlePower": 300,
  
  "simTime": 0,
  "timeStep": 1,
  
  "rsrpServingThreshold": -110,
  "rsrpTargetThreshold": -95,
  "rsrpMeasurementThreshold": -115,
  "dropCallThreshold": 1,
  "latencyThreshold": 50,
  "cpuThreshold": 95,
  "prbThreshold": 95,
  
  "trafficLambda": 30,
  "peakHourMultiplier": 1.5
}

Overwriting app/train_scenarios/high_speed.json


In [3]:
%%writefile app/train_scenarios/rural.json
{
  "name": "3GPP Rural Macro",
  "description": "3GPP Rural deployment scenario with wide area coverage for high speed vehicles",
  "deploymentScenario": "rural",
  
  "carrierFrequency": 700e6,
  "systemBandwidth": 20e6,
  "layout": "hexagonal_grid",
  "isd": 1732,
  
  "numSites": 19,
  "numSectors": 3,
  "antennaHeight": 35,
  "cellRadius": 1000,
  
  "numUEs": 100,
  "userDistribution": "mixed_outdoor_indoor",
  "ueSpeed": 120,
  "indoorRatio": 0.5,
  "outdoorSpeed": 120,
  
  "minTxPower": 20,
  "maxTxPower": 46,
  "basePower": 1200,
  "idlePower": 300,
  
  "simTime": 0,
  "timeStep": 1,
  
  "rsrpServingThreshold": -115,
  "rsrpTargetThreshold": -105,
  "rsrpMeasurementThreshold": -120,
  "dropCallThreshold": 2,
  "latencyThreshold": 100,
  "cpuThreshold": 90,
  "prbThreshold": 90,
  
  "trafficLambda": 10,
  "peakHourMultiplier": 1.2
}

Overwriting app/train_scenarios/rural.json


# Train

In [3]:
%%writefile config.yaml
training_mode: True
use_gpu: False

checkpoint_load_path: "models/zero.pth"
checkpoint_save_path: "models/um_run1.pth"

actor_lr: 0.0001
critic_lr: 0.0003

energy_reward_scale: 10.0
cost_scale: 1.0

entropy_coef: 0.01
gamma: 0.99
lambda_gae: 0.95
clip_epsilon: 0.2
ppo_epochs: 4
batch_size: 1
buffer_size: 4
n_envs: 4
hidden_dim: 256

# Lagrangian PPO settings
lambda_init: 0.0
lambda_lr: 0.05
lambda_max: 100.0
# Target average cost per step (set per-scenario scale). 0 encourages constraint satisfaction.
cost_target: 0.0



Overwriting config.yaml


In [4]:
import yaml
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    num_envs = config['n_envs']

!python3 app/main_run_scenarios_parallel.py --scenarios-dir app/train_scenarios --n-envs $num_envs

/home/huy/anaconda3/envs/mira/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]

=== Running Benchmark Suite (4 scenarios) ===
Scenarios directory: app/train_scenarios/
Using 4 parallel environments per scenario
Loaded scenarios: ['dense_urban', 'high_speed', 'rural', 'urban_macro']


--- Scenario 1/4: dense_urban ---
Loaded scenario: Dense Urban
Loaded scenario: Dense Urban
Loaded scenario: Dense Urban
Created 4 parallel environments
  Scenario: dense_urban
  Action dim: 21
  Obs dim: 280
Loaded scenario: Dense Urban
Skipping scenario dense_urban: simTime=0
Loaded scenario: Dense Urban

Results for dense_urban:
  Energy: 0.000000 kWh
  Drop Rate: 0.00%
  Latency: 0.0 ms
  Handovers: 0 (Success: 0.0%)

--- Scenario 2/4: high_speed ---